In [1]:
import pandas as pd
import numpy as np

# Necesidades de producto de todas las sucursales
deficit_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\deficit.csv')

# Inventario disponible de productos en otras sucursales distintas a la sucursal con deficit
inventario_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\inventario.csv')

# Matriz de traspasos, prioridad de la ruta entre sucursales, y elegibilidad de la sucursal
prioridades_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\matriz_traspasos.csv')

# Valor para decidir a quien se la da prioridad como sucursal (ordenada segun ventas totales, de mayor a menor; revisar orden)
orden_prioridad_df = pd.read_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Documents\TI\Proyectos_BI\Almacen e inventarios\Traspasos\Data sources\sucursales_prioridad.csv')

# Ordenar los déficits por prioridad de sucursal
deficit_df = deficit_df.merge(orden_prioridad_df, on='sucursal')
deficit_df = deficit_df.sort_values(by='orden')

# Establecer el valor máximo de kms de distancia entre sucursales
kms_maximos_distancia = 5000

# Filtrar matriz de traspasos, según kms máximos y elegibilidad
prioridades_df = prioridades_df[(prioridades_df['prioridad'] <= kms_maximos_distancia) & (prioridades_df['elegible'] == 1)]

# Ajustar el inventario disponible para reflejar la relación con el excedente
inventario_df['inventario_ajustado'] = inventario_df['inventario_disponible']
inventario_df['excedente_ajustado'] = inventario_df['excedente']

# Normalización de valores
prioridad_max = prioridades_df['prioridad'].max()  
clasificacion_dict = {'A': 1, 'B': 2, 'C': 3, 'L': 4}
inventario_max = inventario_df['inventario_ajustado'].max()
excedente_max = inventario_df['excedente_ajustado'].max()
clas_max = max(clasificacion_dict.values())

# Asignar los pesos específicos
w_P = 0.10 # Prioridad geográfica    .30
w_C = 0.10 # Prioridad de clasificación ABC
w_I = 0.30 # Prioridad por inventario disponible   .15
w_E = 0.50 # Prioridad por excedente

def calcular_puntuacion(s_origen, s_destino, producto):
    # Obtener datos relevantes
    prioridad = prioridades_df[(prioridades_df['sucursal_origen'] == s_origen) & (prioridades_df['sucursal_destino'] == s_destino)]['prioridad'].values[0]
    clasificacion = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['clasificacion'].values[0]
    inventario_ajustado = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['inventario_ajustado'].values[0]
    excedente = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['excedente_ajustado'].values[0]
    costo_unitario = inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)]['costo_unitario'].values[0]
    
    # Convertir clasificación a un valor numérico
    clasificacion_val = clasificacion_dict[clasificacion]
    
    # Normalización
    P_norm = (prioridad_max - prioridad + 1) / prioridad_max  # Prioridad geográfica menor es mejor (menos kms de distancia)
    C_norm = clasificacion_val / clas_max  # Máximo valor de clasificación ABC
    I_norm = inventario_ajustado / inventario_max
    E_norm = excedente / excedente_max
    
    # Calcular puntuación
    puntuacion = (w_P * P_norm) + (w_C * C_norm) + (w_I * I_norm) + (w_E * E_norm)
    return puntuacion, costo_unitario, clasificacion

def satisfacer_deficit(deficit_row):
    
    sucursal_destino = deficit_row['sucursal']
    producto = deficit_row['producto']
    unidades_necesarias = deficit_row['unidades_necesarias']
    
    # Filtrar sucursales elegibles y calcular puntuaciones
    sucursales_origen = prioridades_df[prioridades_df['sucursal_destino'] == sucursal_destino]['sucursal_origen'].unique()
    puntuaciones = []
    
    for s_origen in sucursales_origen:
        if not inventario_df[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto)].empty:
            puntuacion, costo_unitario, clasificacion = calcular_puntuacion(s_origen, sucursal_destino, producto)
            puntuaciones.append((s_origen, puntuacion, costo_unitario, clasificacion))
                
    # Ordenar sucursales por puntuación descendente
    puntuaciones.sort(key=lambda x: x[1], reverse=True)
    
    # Realizar traspasos hasta satisfacer el déficit
    traspasos = []
    unidades_totales_traspasadas = 0
    for s_origen, _, costo_unitario, clasificacion in puntuaciones:
        if unidades_necesarias <= 0:
            break
        # print(f"Evaluando s_origen: {s_origen}, producto: {producto}, clasificacion: {clasificacion}")

        # Guardar el 80% del excedente inicial si la clasificación es 'A' o 'B'
        if clasificacion in ['A', 'B']:
            excedente_inicial = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'].values[0]
            limite_excedente = int(excedente_inicial * 0.7)
            # print(f"Límite de excedente para 'A' o 'B': {limite_excedente}")
        else:
            limite_excedente = None

        while unidades_necesarias > 0:
            excedente = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'].values[0]
            inventario_ajustado = inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'].values[0]
            # print(f"Excedente: {excedente}, Inventario ajustado: {inventario_ajustado}, Unidades necesarias: {unidades_necesarias}")

            if excedente > 0:
                if clasificacion in ['A', 'B']:
                    unidades_traspaso = min(limite_excedente, unidades_necesarias)
                else:
                    unidades_traspaso = min(excedente, unidades_necesarias)
                # print(f"Unidades traspaso (excedente): {unidades_traspaso}")
                traspasos.append((s_origen, sucursal_destino, producto, unidades_traspaso, costo_unitario))
                inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'excedente_ajustado'] -= unidades_traspaso
                inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'] -= unidades_traspaso
                unidades_necesarias -= unidades_traspaso
                unidades_totales_traspasadas += unidades_traspaso
                # Reducir el límite del excedente por las unidades que se han traspasado
                if clasificacion in ['A', 'B']:
                    limite_excedente -= unidades_traspaso
            elif inventario_ajustado > 0:
                if clasificacion == 'L':
                    unidades_traspaso = min(inventario_ajustado, unidades_necesarias)
                elif clasificacion == 'C':
                    unidades_traspaso = min(int(inventario_ajustado * 0.1), unidades_necesarias)
                else:
                    break
                # print(f"Unidades traspaso (inventario ajustado): {unidades_traspaso}")
                if unidades_traspaso > 0:
                    traspasos.append((s_origen, sucursal_destino, producto, unidades_traspaso, costo_unitario))
                    inventario_df.loc[(inventario_df['sucursal'] == s_origen) & (inventario_df['producto'] == producto), 'inventario_ajustado'] -= unidades_traspaso
                    unidades_necesarias -= unidades_traspaso
                    unidades_totales_traspasadas += unidades_traspaso
                else:
                    break
            else:
                break
            # Añadir una condición de escape para evitar bucles infinitos
            if unidades_traspaso == 0:
                # print("No se pueden transferir más unidades, saliendo del bucle.")
                break

    deficit_residual = deficit_row['unidades_necesarias'] - unidades_totales_traspasadas
    return traspasos, deficit_residual

# Aplicar el proceso para cada déficit
todos_traspasos = []
deficits_residuales = []

for idx, row in deficit_df.iterrows():
    traspasos, deficit_residual = satisfacer_deficit(row)
    todos_traspasos.extend(traspasos)
    if deficit_residual > 0:
        deficits_residuales.append((row['sucursal'], row['producto'], deficit_residual))

# Crear el DataFrame con los traspasos realizados
traspasos_df = pd.DataFrame(todos_traspasos, columns=['sucursal_origen', 'sucursal_destino', 'producto', 'unidades_traspaso', 'costo_unitario'])
traspasos_df = traspasos_df[traspasos_df['unidades_traspaso'] != 0]

# Crear el DataFrame con los déficits residuales
deficits_residuales_df = pd.DataFrame(deficits_residuales, columns=['sucursal_destino', 'producto', 'unidades_no_cubiertas'])

# Agrupar por sucursal_origen, sucursal_destino y producto
traspasos_df = traspasos_df.groupby(['sucursal_origen', 'sucursal_destino', 'producto']).agg({
    'unidades_traspaso': 'sum',
    'costo_unitario': 'mean'
}).reset_index()

traspasos_df = traspasos_df.sort_values(by=['producto', 'sucursal_destino', 'sucursal_origen'], ascending=[True, True, True])

# Exportar traspasos realizados y déficits residuales a archivos CSV
traspasos_df.to_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Desktop\traspasos_recomendados.csv', index=False)
deficits_residuales_df.to_csv(r'C:\Users\diego.salinas\OneDrive - RODAMIENTOS Y ACCESORIOS SA DE CV\Desktop\traspasos_no_procesados.csv', index=False)
print("Listo.")


Listo.
